In [ ]:
import pandas as pd
import numpy as np
from classifier import Classifier

In [ ]:
import os
path0 = 'Rankings2'
path1 = 'Dane'
path2 = 'duw'

path = path1 + '/' + path2

In [ ]:
rul_files = [f for f in os.listdir(path) if f.endswith('.rul')]
# rul_files = ['FLduwb00002U.rul']
display(rul_files)
text_files = [f for f in os.listdir(path) if f.endswith('.csv') and f[1]=='T']
# text_files = ['FT1duwb00002URses.csv','FT2duwb00002URses.csv']
display(text_files)

In [ ]:
#f[0]=='M dla duw
#f[1]=='M dla dsF, dsK, duf
M=[f for f in rul_files if f[0 if path[-1]=='w' else 1]=='M']
F=[f for f in rul_files if f[0]=='F']

MT=[[f for f in text_files if f[0]=='M' and f[-14:-8]==x[-10:-4]] for x in M]
FT=[[f for f in text_files if f[0]=='F' and f[-14:-8]==x[-10:-4]] for x in F]

In [ ]:
print('F =',F)
print('FT =',FT)
print('M =',M)
print('MT =',MT)

### DF creation 

In [ ]:
def create_df_results(testfile: list, classifier: Classifier, revert=False)->pd.DataFrame:
    g1=[(y,x) for y in ['length','support'] for x in ['min','avg','max']]
    g2=[('number of rules','')]
    g3=[('accuracy',x) for x in testfile]
    #row_list = classifier.ranking.index if revert else reversed(classifier.ranking.index)
    row_list = list(reversed(classifier.ranking.index))
    #chodzi o pełen zbiór
    row_list.insert(0,'full')
    col_list = pd.MultiIndex.from_tuples(g1+g2+g3)
    g=pd.DataFrame(np.nan,row_list,col_list)
    return g

### classify func

In [ ]:
#Backward
def classify(classifier: Classifier, results: pd.DataFrame, test_files: list):
    rules_copy=classifier.rules.copy()
    attrs_in_rules=[]
    for attr in classifier.ranking.index:
        #print(attr)
        attrs_in_rules.append(attr)
        
        #wybór reguł zawierających minimum jeden z powyższych atrybutów
        condition=pd.Series(False,classifier.rules.index)
        for y in attrs_in_rules:
            condition=condition|pd.notna(classifier.rules[y]) 
        classifier.rules=classifier.rules.loc[condition]
        
        #length
        length=classifier.rules.rule.apply(lambda x: len(x))
        results.at[attr,('length','min')]=min(length, default=0)
        results.at[attr,('length','avg')]=sum(length)/len(length) if len(length) else 0
        results.at[attr,('length','max')]=max(length, default=0)
        #support
        support=classifier.rules.support
        results.at[attr,('support','min')]=min(support, default=0)
        results.at[attr,('support','avg')]=sum(support)/len(support) if len(support) else 0
        results.at[attr,('support','max')]=max(support, default=0)
        #number of rules 
        results.at[attr,('number of rules','')]=len(classifier.rules)
        
        for test_file in test_files:
            classifier.loadTestSet(path+'/'+test_file)
            
            (cm, acc)=classifier.classify()
            #acc=sum([cm[y][y1] for y,y1 in zip(cm.columns,cm.index)])/len(classifier.test_df)
            
            #accuracy
            results.at[attr,('accuracy',test_file)]=acc
        
        #powrót do pełnego zestawu reguł
        classifier.rules=rules_copy
    return results

In [ ]:
#Forward
def classify2(classifier: Classifier, results: pd.DataFrame, test_files: list):
    rules_copy=classifier.rules.copy()
    ranking=reversed(classifier.ranking.index)
    not_in_ranking = list(set(classifier.rules.columns[:-3]) - set(classifier.ranking.index))
    #TODO: DUW3 pełen zbiór
    # klasyfikacja dla pełnego zbioru
    for attr in ['full']:
        #length
        length=classifier.rules.rule.apply(lambda x: len(x))
        results.at[attr,('length','min')]=min(length, default=0)
        results.at[attr,('length','avg')]=sum(length)/len(length) if len(length) else 0
        results.at[attr,('length','max')]=max(length, default=0)
        #support
        support=classifier.rules.support
        results.at[attr,('support','min')]=min(support, default=0)
        results.at[attr,('support','avg')]=sum(support)/len(support) if len(support) else 0
        results.at[attr,('support','max')]=max(support, default=0)
        #number of rules 
        results.at[attr,('number of rules','')]=len(classifier.rules)
        
        for test_file in test_files:
            classifier.loadTestSet(path+'/'+test_file)
            
            (cm, acc)=classifier.classify()
            
            #accuracy
            results.at[attr,('accuracy',test_file)]=acc
            
    # odfiltrowanie atrybutów nie występujących w rankingu
    for attr in not_in_ranking:
        classifier.rules=classifier.rules.loc[lambda x:pd.isna(x[attr])]
    
    for attr in ranking:
        #length
        length=classifier.rules.rule.apply(lambda x: len(x))
        results.at[attr,('length','min')]=min(length, default=0)
        results.at[attr,('length','avg')]=sum(length)/len(length) if len(length) else 0
        results.at[attr,('length','max')]=max(length, default=0)
        #support
        support=classifier.rules.support
        results.at[attr,('support','min')]=min(support, default=0)
        results.at[attr,('support','avg')]=sum(support)/len(support) if len(support) else 0
        results.at[attr,('support','max')]=max(support, default=0)
        #number of rules 
        results.at[attr,('number of rules','')]=len(classifier.rules)
        
        for test_file in test_files:
            classifier.loadTestSet(path+'/'+test_file)
            
            (cm, acc)=classifier.classify()
            
            #accuracy
            results.at[attr,('accuracy',test_file)]=acc
        
        #odfiltrowanie ostatniego atrybutu // zostawia tylko te reguły które nie zawierają tego atrybutu
        classifier.rules=classifier.rules.loc[lambda x:pd.isna(x[attr])]
    
    classifier.rules=rules_copy
    return results

### New Ranking

In [ ]:
def loadRanking(filepath: str):
    with open(filepath, 'r') as plik:
        lista = [line.strip() for line in plik]

    df = pd.DataFrame({'': lista})
    df.set_index('', inplace=True)
    return df

In [ ]:
def resultsFix(results: pd.DataFrame,atrn: int) -> pd.DataFrame:
    attrs=list(range(len(results), 0, -1))
    attrs[0]=atrn
    results.insert(0,'number of attributes',attrs)
    results['mean']=results['accuracy'].mean(axis=1)
    return results

### SAVE TO xlsx

In [ ]:
from openpyxl import load_workbook

def df_to_excel(df,filepath,sheet_name):
    # Otwórz istniejący plik Excel
    workbook = load_workbook(filepath)

    # Konwertuj DataFrame na arkusz Excel
    writer = pd.ExcelWriter(filepath, engine='openpyxl') 
    writer.book = workbook

    # Dopisz DataFrame do nowego arkusza
    df.to_excel(writer, sheet_name=sheet_name)

    # Zapisz zmiany w pliku
    writer.save()


### Classification

#### Preparation

In [ ]:
classifier = Classifier()
# False = backward 
# True = forward
reverts=[True]
#algotithms=['oner','greedy','reducts']
algotithms=['greedy']

#### All

In [ ]:

for alg in algotithms:
    for revert in reverts:
        # MALE
        for rulefile, test_files in zip(M,MT):
            print(alg+' ',revert,' '+rulefile, ' is processing...')
            
            #Wczytanie reguł
            classifier.loadRuleSet(path+'/'+rulefile)
            
            #Wczytanie rankingu
            if alg=='oner':
                classifier.ranking=loadRanking('Rankings/RankingOnerMale.txt')
            elif alg=='greedy':
                classifier.loadRanking(path0+'/'+path2+'/'+rulefile[:-4]+'Rses.ranking')
            elif alg=='reducts':
                classifier.ranking=loadRanking('Rankings/RankingReductsMale.txt')
            else:
                print('sth wrong')
                    
            #Stworzenie DF wynikowego
            results=create_df_results(test_files,classifier,revert)
            
            #Przeprowadzenie klasyfikacji i obrobienie wyników
            results=classify2(classifier,results,test_files) if revert else classify(classifier,results,test_files)
            results=resultsFix(results,classifier.ATRN)
            
            #Zapisy do plików
            df_to_excel(results,'Results2/Results_Male.xlsx',sheet_name=path2+rulefile[-6:-5])
            
        # FEMALE
        for rulefile, test_files in zip(F,FT):
            print(alg+' ',revert,' '+rulefile, ' is processing...')
            
            #Wczytanie reguł
            classifier.loadRuleSet(path+'/'+rulefile)
            
            #Wczytanie rankingu
            if alg=='oner':
                classifier.ranking=loadRanking('Rankings/RankingOnerFemale.txt')
            elif alg=='greedy':
                classifier.loadRanking(path0+'/'+path2+'/'+rulefile[:-4]+'Rses.ranking')
            elif alg=='reducts':
                classifier.ranking=loadRanking('Rankings/RankingReductsFemale.txt')
            else:
                print('sth wrong')
            
            #Stworzenie DF wynikowego
            results=create_df_results(test_files,classifier,revert)
            
            #Przeprowadzenie klasyfikacji i obrobienie wyników
            results=classify2(classifier,results,test_files) if revert else classify(classifier,results,test_files)
            results=resultsFix(results,classifier.ATRN)
            
            #Zapisy do plików
            df_to_excel(results,'Results2/Results_Female.xlsx',sheet_name=path2+rulefile[-6:-5])